In [1]:
!pip install tabpfn --no-index --find-links=file:///kaggle/input/pip-packages-icr/pip-packages
!pip install ipywidgets
!mkdir -p /opt/conda/lib/python3.10/site-packages/tabpfn/models_diff
!cp /kaggle/input/pip-packages-icr/pip-packages/prior_diff_real_checkpoint_n_0_epoch_100.cpkt /opt/conda/lib/python3.10/site-packages/tabpfn/models_diff/

Looking in links: file:///kaggle/input/pip-packages-icr/pip-packages
mkdir: /opt/conda: Permission denied
cp: directory /opt/conda/lib/python3.10/site-packages/tabpfn/models_diff does not exist


In [2]:
# Import Joblib Module from Scikit Learn
import joblib

import numpy as np                       # NumPy for numerical computations
import pandas as pd                      # Pandas for data manipulation and analysis
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, normalize   # LabelEncoder for encoding categorical variables, normalize for feature scaling
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier   # GradientBoostingClassifier and RandomForestClassifier for classification models
from sklearn.metrics import accuracy_score   # accuracy_score for evaluating model performance
from sklearn.impute import SimpleImputer   # SimpleImputer for handling missing values
import imblearn   # imblearn for imbalanced dataset handling
from imblearn.over_sampling import RandomOverSampler   # RandomOverSampler for oversampling minority class
from imblearn.under_sampling import RandomUnderSampler   # RandomUnderSampler for undersampling majority class
import xgboost   # XGBoost for gradient boosting models
import inspect   # inspect for retrieving information about live objects
from collections import defaultdict   # defaultdict for creating a dictionary with default values
from tabpfn import TabPFNClassifier   # TabPFNClassifier for a specific classification model
import warnings   # warnings for ignoring warnings during runtime
from sklearn.model_selection import KFold as KF
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm
import sys
import os

/Users/hadangvu/anaconda3/lib/python3.10/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Load Dataset

In [4]:
train = pd.read_csv('../datasets/icr-identify-age-related-conditions/train.csv')
train = train.drop(['Id','EJ'], axis=1)
columns = train.columns
imputer = SimpleImputer(missing_values = np.nan, strategy ='median')
imputer = imputer.fit(train)
train = imputer.transform(train)
train = pd.DataFrame(train, columns = columns)
train.isnull().sum()

AB       0
AF       0
AH       0
AM       0
AR       0
AX       0
AY       0
AZ       0
BC       0
BD       0
BN       0
BP       0
BQ       0
BR       0
BZ       0
CB       0
CC       0
CD       0
CF       0
CH       0
CL       0
CR       0
CS       0
CU       0
CW       0
DA       0
DE       0
DF       0
DH       0
DI       0
DL       0
DN       0
DU       0
DV       0
DY       0
EB       0
EE       0
EG       0
EH       0
EL       0
EP       0
EU       0
FC       0
FD       0
FE       0
FI       0
FL       0
FR       0
FS       0
GB       0
GE       0
GF       0
GH       0
GI       0
GL       0
Class    0
dtype: int64

## Cross Validation

In [20]:
NUM_FOLDS = 5
skf = StratifiedKFold(n_splits=NUM_FOLDS, shuffle=True, random_state=42)
splitter = skf.split(train, train.Class)

for fold_idx, (train_idx, val_idx) in enumerate(splitter):
    print(f'Getting fold number {fold_idx}')
    df_train = train.iloc[train_idx]
    df_val = train.iloc[val_idx]
    
    #drop column Id & reset index
    
#     df_train = df_train.drop(['Id'], axis=1)
    df_train = df_train.reset_index(drop = True)
#     df_val = df_val.drop(['Id'], axis=1)
    df_val = df_val.reset_index(drop = True)
    
    #kfold path
    save_dir = f'../datasets/kfold/fold{fold_idx}'
    os.makedirs(save_dir, exist_ok = True)
    
    #saving
    df_train.to_csv(os.path.join(save_dir, 'train.csv'), index = False)
    df_val.to_csv(os.path.join(save_dir, 'val.csv'), index = False)
    
    # for testing
    save_dir1 = f'../datasets/kfold1/fold{fold_idx}'
    os.makedirs(save_dir1, exist_ok = True)
    df_train = df_train.reset_index(drop=True)
    df_val = df_val.reset_index(drop=True)
    df_train.to_csv(os.path.join(save_dir1, 'train.csv'))
    df_val.to_csv(os.path.join(save_dir1, 'val.csv'))

Getting fold number 0
Getting fold number 1
Getting fold number 2
Getting fold number 3
Getting fold number 4


## Pre-processing

In [21]:
def prepair_input(df, classi):
    columns = df.columns
    
    # Convert the values in the 'EJ' column of the 'test' dataframe to binary values (0 or 1),
    # based on the occurrence of the 'first_category' in the 'train' dataframe
#     first_category = df.EJ.unique()[0]
#     df.EJ = df.EJ.eq(first_category).astype('int')

    df = df.rename(columns={'BD ': 'BD', 'CD ': 'CD', 'CW ': 'CW', 'FD ': 'FD'})
    
    imputer = SimpleImputer(missing_values = np.nan, strategy ='median')
    imputer = imputer.fit(df)
    df = imputer.transform(df)
    df = pd.DataFrame(df, columns = columns)
    
    # Create a RandomOverSampler object with a random state of 42
    ros = RandomOverSampler(random_state=42)

    # Resample the 'train_pred_and_time' dataframe and 'greeks.Alpha' series using RandomOverSampler
    # The resampled data is assigned to 'train_ros' and 'y_ros' respectively
    x_ros, y_ros = ros.fit_resample(df, classi)
    return x_ros, y_ros

def normolized(df):
    columns = df.columns
    
    scaler = StandardScaler()
    model = scaler.fit(df)
    scaled_df = model.transform(df)
    
    scaled_df = pd.DataFrame(scaled_df, columns = columns)
    return scaled_df

## Balanced Log Loss

In [22]:
def balanced_log_loss(y_true, y_pred):
    # y_true: correct labels 0, 1
    # y_pred: predicted probabilities of class=1
    # calculate the number of observations for each class
    N_0 = np.sum(1 - y_true)
    N_1 = np.sum(y_true)
    # calculate the weights for each class to balance classes
    w_0 = 1 / N_0
    w_1 = 1 / N_1
    # calculate the predicted probabilities for each class
    p_0 = np.clip(y_pred[:, 0], 1e-15, 1 - 1e-15)
    p_1 = np.clip(y_pred[:, 1], 1e-15, 1 - 1e-15)
    # calculate the summed log loss for each class
    log_loss_0 = -np.sum((1 - y_true) * np.log(p_0))
    log_loss_1 = -np.sum(y_true * np.log(p_1))
    # calculate the weighted summed logarithmic loss
    # (factgor of 2 included to give same result as LL with balanced input)
    balanced_log_loss = 2*(w_0 * log_loss_0 + w_1 * log_loss_1) / (w_0 + w_1)
    # return the average log loss
    return balanced_log_loss/(N_0+N_1)

## Model

In [23]:
class Ensemble():
    def __init__(self):
        self.classifiers =[xgboost.XGBClassifier(n_estimators=100,max_depth=3,learning_rate=0.2,subsample=0.9,colsample_bytree=0.85),
                           xgboost.XGBClassifier(),
                           TabPFNClassifier(N_ensemble_configurations=12),
                           TabPFNClassifier(N_ensemble_configurations=12),
                           RandomForestClassifier(n_estimators=100, max_depth=3, random_state=0)]
    def fit(self,X,y):
        for i, classifier in enumerate(self.classifiers):
            if i in [2, 3]:
                classifier.fit(X,y,overwrite_warning =True)
            else :
                classifier.fit(X, y)
     
    def predict_proba(self, x):
        # N_models * N_rows * N_classes (4 * 5 * 2)
        probabilities = np.stack([classifier.predict_proba(x) for classifier in self.classifiers])
        averaged_probabilities = np.mean(probabilities, axis=0) # N_rows * N_classes
        return averaged_probabilities

# Post processing

In [24]:
def calibrate_prob(probs):
    print('TYPE:', probs.shape, type(probs))
    ret = probs.copy()
    ret[probs > 0.80] = 1.0
    ret[probs < 0.20] = 0.0
    return ret

## Training

In [41]:
def training(model):
    outer_results = list()   # List to store the loss results of each outer fold
    best_loss = np.inf   # Variable to track the best loss achieved during training
    split = 0   # Counter for the current split
    splits = 5   # Total number of splits for the inner cross-validation
    models = []   # List to store the trained models for each inner fold

    # Loop over the splits of the inner cross-validation using tqdm for progress visualization
    for split in range(splits):
        print('fold', split)
        #loading train & test dataset for each fold
        save_dir = f'../datasets/kfold/fold{split}'
        
        # x_train & y_train
        df_train = pd.read_csv(os.path.join(save_dir, 'train.csv'))
        x_train = df_train.drop(['Class'], axis=1)
        y_train = df_train.Class
        # pre-processing
        x_train, y_train = prepair_input(x_train, y_train)

        # x_val & y_val
        df_val = pd.read_csv(os.path.join(save_dir, 'val.csv'))
        x_val = df_val.drop(['Class'], axis=1)
        y_val = df_val.Class

        #fitting model
        model.fit(x_train, y_train)   # Fit the model on the training data
        models.append(model)   # Append the trained model to the list of models

        y_pred = model.predict_proba(x_val)   # Predict probabilities for the validation set   # Calculate class probabilities
        y_p = np.empty((y_pred.shape[0],y_pred.shape[1]))   # Create an empty array to store predicted labels
        
        print(y_pred)
        print('sub', y_pred[:, 0:].sum(axis=1))
        
        y_p = calibrate_prob(y_pred)
        
        loss = balanced_log_loss(y_val, y_p)  # Calculate the balanced log loss between the predicted labels and the true labels

        # checking
        y_val = y_val.to_frame()
        y_val.rename(columns = {'Class': 'gt'}, inplace = True)
        y_val['pred'] = y_p[:, 1]
#         print(type(y_val['gt']), type(y_val.loc[0, 'gt']), type(y_val['pred']), type(y_val.loc[0, 'pred']))
        p00 = y_pred[:, 1]
        p00 = p00.flatten()
        y_val['prob'] = p00
        display(y_val)

        if loss < best_loss:
            best_model = model   # Save the best model based on the lowest loss
            best_loss = loss   # Update the best loss
            print('best_model_saved')

        outer_results.append(loss)   # Append the loss to the list of outer results
        print('>val_loss=%.5f, split = %.1f' % (loss, split))

        split += 1

    print('LOSS: %.5f' % (np.mean(outer_results)))  # Print the average loss across all outer folds
    print('Best loss', best_loss)
    return best_model, models   # Return the best model and the list of trained models

In [42]:
yt = Ensemble()

m, models = training(yt)

Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
fold 0
[09:33:10] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[09:33:10] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Users/hadangvu/anaconda3/lib/python3.10/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/hadangvu/anaconda3/lib/python3.10/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Users/hadangvu/anaconda3/lib/python3.10/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass 

[[0.69291046 0.30708957]
 [0.9740727  0.02592732]
 [0.94006551 0.0599345 ]
 [0.32252747 0.67747255]
 [0.9170994  0.08290062]
 [0.90582596 0.09417402]
 [0.90066061 0.09933942]
 [0.91603931 0.08396069]
 [0.96211108 0.03788894]
 [0.93560454 0.06439548]
 [0.95167473 0.04832526]
 [0.90592486 0.09407515]
 [0.97754406 0.02245593]
 [0.90971462 0.09028537]
 [0.96720359 0.03279642]
 [0.86784238 0.13215762]
 [0.96135796 0.03864205]
 [0.96017498 0.03982501]
 [0.90974692 0.09025307]
 [0.95809404 0.04190596]
 [0.96443021 0.03556979]
 [0.46831636 0.53168363]
 [0.9578133  0.04218668]
 [0.94017593 0.05982407]
 [0.95002923 0.04997076]
 [0.25090724 0.74909276]
 [0.52855132 0.47144869]
 [0.87944081 0.12055921]
 [0.1171993  0.88280071]
 [0.95018624 0.04981381]
 [0.90288671 0.09711328]
 [0.96157677 0.03842325]
 [0.95930863 0.04069141]
 [0.84891057 0.15108941]
 [0.2332335  0.76676651]
 [0.81551972 0.18448029]
 [0.92587274 0.07412726]
 [0.93142077 0.06857921]
 [0.95805044 0.04194957]
 [0.02956497 0.97043504]


,gt,pred,prob
0,1.0,0.307090,0.307090
1,0.0,0.000000,0.025927
2,0.0,0.000000,0.059935
3,1.0,0.677473,0.677473
4,0.0,0.000000,0.082901
5,0.0,0.000000,0.094174
6,0.0,0.000000,0.099339
7,0.0,0.000000,0.083961
8,0.0,0.000000,0.037889
9,0.0,0.000000,0.064395


best_model_saved
>val_loss=0.21344, split = 0.0
fold 1
[09:33:14] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[09:33:14] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


...

>val_loss=1.60513, split = 3.0
fold 4
[09:33:26] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[09:33:26] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Users/hadangvu/anaconda3/lib/python3.10/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/hadangvu/anaconda3/lib/python3.10/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Users/hadangvu/anaconda3/lib/python3.10/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass 

[[0.95023419 0.0497658 ]
 [0.95472109 0.04527892]
 [0.96524161 0.03475836]
 [0.92753473 0.0724653 ]
 [0.93090871 0.06909129]
 [0.14518335 0.85481666]
 [0.89017739 0.10982259]
 [0.8701304  0.1298696 ]
 [0.90920162 0.09079837]
 [0.81010187 0.18989815]
 [0.94746665 0.05253336]
 [0.95611836 0.04388167]
 [0.95443162 0.04556839]
 [0.93192106 0.06807893]
 [0.95525915 0.04474082]
 [0.07340821 0.92659179]
 [0.95766476 0.04233526]
 [0.93263615 0.06736386]
 [0.92462372 0.07537627]
 [0.92757154 0.07242845]
 [0.94895019 0.05104983]
 [0.97313567 0.02686433]
 [0.97383313 0.02616687]
 [0.95303029 0.04696971]
 [0.23296466 0.76703533]
 [0.94431663 0.05568334]
 [0.80085485 0.19914517]
 [0.93319735 0.06680267]
 [0.66825557 0.33174443]
 [0.97029696 0.02970304]
 [0.3176525  0.68234746]
 [0.85004656 0.14995342]
 [0.95037086 0.04962915]
 [0.94797512 0.0520249 ]
 [0.78338484 0.21661516]
 [0.74035903 0.25964097]
 [0.94110528 0.05889471]
 [0.95367319 0.0463268 ]
 [0.93767776 0.06232226]
 [0.95478488 0.04521512]


,gt,pred,prob
0,0.0,0.000000,0.049766
1,0.0,0.000000,0.045279
2,0.0,0.000000,0.034758
3,0.0,0.000000,0.072465
4,0.0,0.000000,0.069091
5,1.0,1.000000,0.854817
6,0.0,0.000000,0.109823
7,0.0,0.000000,0.129870
8,0.0,0.000000,0.090798
9,0.0,0.000000,0.189898


best_model_saved
>val_loss=0.10561, split = 4.0
LOSS: 0.79281
Best loss 0.10561440489188031
